In [2]:
import os
os.chdir('..')

In [25]:
%load_ext autoreload
%autoreload 2
from comm_agents.data.reference_experiments_with_qa import RefExperimentMass
from ipywidgets import interact
import numpy as  np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# experiment parameters
GOLF_HOLE_LOC = 0.1
PARAM_DICT_M = dict(
    m=[2e-12, 1e-12],
    m_ref=2e-12,
    v_ref=1,
    N=100,
    alpha=[0, 0],
    dt=.01,
    gravity=True)

In [33]:
alpha_range = np.linspace(0, 0.6, 20)
m_range = np.linspace(1e-12, 3e-12, 20)
m_range

array([1.00000000e-12, 1.10526316e-12, 1.21052632e-12, 1.31578947e-12,
       1.42105263e-12, 1.52631579e-12, 1.63157895e-12, 1.73684211e-12,
       1.84210526e-12, 1.94736842e-12, 2.05263158e-12, 2.15789474e-12,
       2.26315789e-12, 2.36842105e-12, 2.47368421e-12, 2.57894737e-12,
       2.68421053e-12, 2.78947368e-12, 2.89473684e-12, 3.00000000e-12])

In [37]:
def get_re1_plot(alpha1, alpha2, m1, m2):
    rem = RefExperimentMass(**PARAM_DICT_M)
    rem.set_initial_state()
    rem.alpha = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    rem.run()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC)
    
interact(get_re1_plot, alpha1=alpha_range, alpha2=alpha_range, m1=m_range, m2=m_range)

interactive(children=(Dropdown(description='alpha1', options=(0.0, 0.031578947368421054, 0.06315789473684211, …

<function __main__.get_re1_plot(alpha1, alpha2, m1, m2)>